# 한국어 텍스트 감정 분석 - 최적화된 파인튜닝

이 노트북은 WandB Sweep을 통해 찾은 최적의 하이퍼파라미터를 사용하여 최종 모델을 훈련합니다.

## 주요 기능
- 최적화된 하이퍼파라미터를 사용한 파인튜닝
- DAPT (Domain-Adapted Pre-Training) 모델 사용
- 데이터셋 비율 조정: original(1.0) : augment(3.0)
- 모델 저장 및 평가
- WandB를 통한 실험 추적


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed
)

# PEFT for LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
# 환경 설정
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()

RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")


디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [ ]:
import datetime
now = datetime.datetime.now()
TIMESTAMP = now.strftime("%Y-%m-%d_%H-%M-%S")

# 고정 설정
PROJECT_NAME = "[domain_project]_Final_Training_original"
MODEL_NAME = "TAPT_klue_Roberta-kor-base_final_training_2025-10-30_00-40-22_RANDOM_42"
RUN_NAME = f"{MODEL_NAME}_final_training_{TIMESTAMP}_RANDOM_{RANDOM_STATE}"

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [4]:
# 최적화된 하이퍼파라미터 설정 (Sweep 결과 기반)
class OptimalConfig:
    # 최적화된 하이퍼파라미터 (Sweep에서 찾은 최고 성능)
    learning_rate = 1e-05  
    batch_size = 400
    label_smoothing_factor = 0.12  
    weight_decay = 0.04  
    num_epochs = 4
    augment_ratio = 0.0  # original 대비 3배

    
    # 데이터셋 비율 설정
    original_ratio = 1.0  # 항상 1.0 (기준)
    
    # 기타 설정
    warmup_steps = 10
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    early_stopping_patience = 2
    save_best_model = True  # 최고 성능 모델 저장
    
    # 모델 저장 경로
    save_model_path = f"./Final_Models/original/{RUN_NAME}"
    
    # WandB 설정
    use_wandb = True
    project_name = PROJECT_NAME

optimal_config = OptimalConfig()

print("✅ 최적화된 하이퍼파라미터 설정 완료")
print(f"learning_rate: {optimal_config.learning_rate:.2e}")
print(f"batch_size: {optimal_config.batch_size}")
print(f"label_smoothing_factor: {optimal_config.label_smoothing_factor:.3f}")
print(f"weight_decay: {optimal_config.weight_decay:.3f}")
print(f"num_epochs: {optimal_config.num_epochs}")
print(f"augment_ratio: {optimal_config.augment_ratio:.1f}")
print(f"모델 저장 경로: {optimal_config.save_model_path}")


✅ 최적화된 하이퍼파라미터 설정 완료
learning_rate: 1.00e-05
batch_size: 400
label_smoothing_factor: 0.120
weight_decay: 0.040
num_epochs: 4
augment_ratio: 0.0
모델 저장 경로: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42


In [5]:
# 고정 설정
class FixedConfig:
    # 모델 설정
    model_name = MODEL_NAME
    base_models_path = ""
    local_model_path = base_models_path + MODEL_NAME

    is_DAPT = True
    DAPT_model_path = f"/data/ephemeral/home/code/Final_Models/{MODEL_NAME}"
    
    num_classes = 4
    max_length = 128

    # 데이터 설정
    train_data_path = "/data/ephemeral/home/code/data/train_final_augX3.csv"
    val_data_path = "/data/ephemeral/home/code/data/val_final.csv"
    
    # 고정 훈련 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Loss 설정
    loss_function = "CrossEntropy"
    use_label_smoothing = True
    use_lora = False

fixed_config = FixedConfig()
print("✅ 고정 설정 완료")


✅ 고정 설정 완료


In [6]:
# 데이터셋 비율에 따른 샘플링 함수
def sample_dataset_by_ratio(df, augment_ratio=1.0, random_state=42):
    """
    데이터셋을 비율에 따라 샘플링하는 함수
    
    Args:
        df: 전체 데이터프레임
        augment_ratio: augment 데이터 비율 (original 대비)
        random_state: 랜덤 시드
    
    Returns:
        sampled_df: 샘플링된 데이터프레임
    """
    np.random.seed(random_state)
    
    # 각 타입별 데이터 분리
    original_data = df[df['type'] == 'original'].copy()
    augment_data = df[df['type'] == 'augment'].copy()
    
    print(f"원본 데이터 분포:")
    print(f"  Original: {len(original_data):,}개")
    print(f"  Augment: {len(augment_data):,}개")
    
    # Original 데이터는 항상 100% 사용
    sampled_original = original_data.copy()
    
    # Augment 데이터 샘플링
    if augment_ratio > 0 and len(augment_data) > 0:
        # original 대비 비율로 샘플링
        target_augment_size = int(len(original_data) * augment_ratio)
        if target_augment_size > len(augment_data):
            # 요청된 크기가 전체보다 크면 전체 사용
            sampled_augment = augment_data.copy()
        else:
            # 비율에 맞게 샘플링
            sampled_augment = augment_data.sample(n=target_augment_size, random_state=random_state)
    else:
        sampled_augment = pd.DataFrame(columns=df.columns)

    
    # 샘플링된 데이터 합치기
    sampled_df = pd.concat([sampled_original, sampled_augment], ignore_index=True)
    
    print(f"\\n샘플링된 데이터 분포:")
    print(f"  Original: {len(sampled_original):,}개 (비율: 1.0)")
    print(f"  Augment: {len(sampled_augment):,}개 (비율: {augment_ratio:.1f})")
    print(f"  총 데이터: {len(sampled_df):,}개")
    
    return sampled_df

print("✅ 데이터셋 샘플링 함수 정의 완료")


✅ 데이터셋 샘플링 함수 정의 완료


In [7]:
# 전체 데이터 로드 (비율 샘플링을 위해)
print("전체 데이터 로드 중...")

# 원본 데이터 로드 (preprocessing_final.ipynb에서 생성된 데이터)
full_df = pd.read_csv(fixed_config.train_data_path)
print(f"전체 데이터: {len(full_df):,}개")


# 검증 데이터 로드 (original만 사용)
val_df = pd.read_csv(fixed_config.val_data_path)

full_df = pd.concat([full_df, val_df], ignore_index=True)
print(f"검증 데이터: {len(val_df):,}개")
print(f"전체 데이터: {len(full_df):,}개")
print("✅ 데이터 로드 완료")


전체 데이터 로드 중...
전체 데이터: 536,966개
검증 데이터: 6,823개
전체 데이터: 543,789개
✅ 데이터 로드 완료


In [8]:
# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    """
    리뷰 텍스트 데이셋 클래스
    - BERT 모델 훈련/추론을 위한 PyTorch Dataset 구현
    - 텍스트 토크나이징 및 텐서 변환 처리
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        데이터셋 초기화
        """
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터 아이템 반환
        """
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

print("✅ 데이터셋 클래스 정의 완료")


✅ 데이터셋 클래스 정의 완료


In [9]:
# 평가 메트릭 함수
def compute_metrics(predictions, labels):
    """
    모델 평가 메트릭 계산 함수
    """
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
    }

print("✅ 평가 메트릭 함수 정의 완료")


✅ 평가 메트릭 함수 정의 완료


In [10]:
def model_load_from_local(local_model_path : str = None, model_name : str = None):
    # 1. 토크나이저 로드
    # 절대 경로를 사용하여 로컬 모델 로드
    print("로컬 경로에서 토크나이저 로딩 중...")

    # 경로가 존재하는지 확인
    if not os.path.exists(local_model_path):
        print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
        return None, None
    else:
        print(f"✅ 경로 확인됨: {local_model_path}")

    tokenizer = AutoTokenizer.from_pretrained(local_model_path)

    # 2. 모델 로드
    # 마찬가지로 로컬 경로(local_model_path)를 사용
    print("로컬 경로에서 모델 로딩 중...")
    model = AutoModelForSequenceClassification.from_pretrained(
        local_model_path,
        num_labels=4,
        ignore_mismatched_sizes=True
    )

    print("✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!")

    return model, tokenizer


In [11]:
import torch.nn.functional as F

def label_smoothing_cross_entropy(predictions, targets, smoothing=0.1):
    """
    predictions: 모델 출력 (logits) [batch_size, num_classes]
    targets: 정답 라벨 (정수) [batch_size]
    smoothing: smoothing factor
    """
    num_classes = predictions.size(-1)
    
    # 하드 라벨을 원-핫으로 변환
    true_dist = torch.zeros_like(predictions)
    true_dist.fill_(smoothing / (num_classes - 1))
    true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - smoothing)
    
    # KL divergence = Cross Entropy with smoothed labels
    return F.kl_div(F.log_softmax(predictions, dim=1), true_dist, reduction='batchmean')

print("✅ Label Smoothing 함수 정의 완료")


✅ Label Smoothing 함수 정의 완료


In [12]:
def train_epoch(model, dataloader, optimizer, scheduler, scaler, device, config, criterion_fn):
    model.train()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for step, batch in enumerate(progress_bar):
        # 배치를 디바이스로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        # 혼합 정밀도 훈련
        if scaler is not None:
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                logits = outputs.logits
                loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
        else:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            logits = outputs.logits
            loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
            
            loss.backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        
        total_loss += loss.item() * config.gradient_accumulation_steps
        
        # 예측값 저장
        predictions.extend(logits.detach().cpu().numpy())
        labels.extend(batch_labels.detach().cpu().numpy())
        
        # 진행률 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item() * config.gradient_accumulation_steps:.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics


In [13]:
# 검증 함수
def validate_epoch(model, dataloader, device, criterion_fn):
    """한 에포크 검증"""
    model.eval()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Validation")
    
    with torch.no_grad():
        for batch in progress_bar:
            # 배치를 디바이스로 이동
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)
            
            # 순전파
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            loss = criterion_fn(logits, batch_labels)
            
            total_loss += loss.item()
            
            # 예측값 저장
            predictions.extend(logits.detach().cpu().numpy())
            labels.extend(batch_labels.detach().cpu().numpy())
            
            # 진행률 업데이트
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

print("✅ 검증 함수 정의 완료")


✅ 검증 함수 정의 완료


In [14]:
# 모델 저장 함수
def save_model(model, tokenizer, save_path, epoch, val_accuracy):
    """모델과 토크나이저를 저장하는 함수"""
    os.makedirs(save_path, exist_ok=True)
    
    # 모델 저장
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    # 설정 정보 저장
    config_info = {
        "epoch": epoch,
        "val_accuracy": val_accuracy,
        "timestamp": TIMESTAMP,
        "model_name": fixed_config.model_name,
        "learning_rate": optimal_config.learning_rate,
        "batch_size": optimal_config.batch_size,
        "label_smoothing_factor": optimal_config.label_smoothing_factor,
        "weight_decay": optimal_config.weight_decay,
        "augment_ratio": optimal_config.augment_ratio,
    }
    
    import json
    with open(os.path.join(save_path, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_info, f, indent=2, ensure_ascii=False)
    
    print(f"✅ 모델 저장 완료: {save_path}")
    print(f"   Epoch: {epoch}, Val Accuracy: {val_accuracy:.4f}")

# 메인 훈련 함수
def train_final_model():
    # WandB 초기화
    if optimal_config.use_wandb:
        run = wandb.init(
            project=optimal_config.project_name,
            name=RUN_NAME,
            config={
                "learning_rate": optimal_config.learning_rate,
                "batch_size": optimal_config.batch_size,
                "label_smoothing_factor": optimal_config.label_smoothing_factor,
                "weight_decay": optimal_config.weight_decay,
                "num_epochs": optimal_config.num_epochs,
                "augment_ratio": optimal_config.augment_ratio,
                "model_name": fixed_config.model_name,
                "is_DAPT": fixed_config.is_DAPT
            }
        )
    
    print(f"\n{'='*60}")
    print(f"최종 모델 훈련 시작")
    print(f"learning_rate: {optimal_config.learning_rate:.2e}")
    print(f"batch_size: {optimal_config.batch_size}")
    print(f"label_smoothing_factor: {optimal_config.label_smoothing_factor:.3f}")
    print(f"weight_decay: {optimal_config.weight_decay:.3f}")
    print(f"num_epochs: {optimal_config.num_epochs}")
    print(f"augment_ratio: {optimal_config.augment_ratio:.1f}")
    print(f"모델 저장 경로: {optimal_config.save_model_path}")
    print(f"{'='*60}")
    
    try:
        # 모델 및 토크나이저 로드
        if fixed_config.is_DAPT:
            model, tokenizer = model_load_from_local(fixed_config.DAPT_model_path, fixed_config.model_name)
        else:
            model, tokenizer = model_load_from_local(fixed_config.local_model_path, fixed_config.model_name)
        
        if model is None or tokenizer is None:
            print("❌ 모델 로드 실패")
            return
        
        model = model.to(fixed_config.device)
        
        # 비율에 따라 훈련 데이터 샘플링
        print("\n데이터셋 비율에 따른 샘플링 중...")
        train_df_sampled = sample_dataset_by_ratio(
            full_df, 
            augment_ratio=optimal_config.augment_ratio, 
            random_state=RANDOM_STATE
        )
        
        # 데이터셋 및 데이터로더 생성
        train_dataset = ReviewDataset(
            train_df_sampled["review"],
            train_df_sampled["label"],
            tokenizer,
            fixed_config.max_length,
        )

        val_dataset = ReviewDataset(
            val_df["review"],
            val_df["label"],
            tokenizer,
            fixed_config.max_length,
        )

        train_dataloader = DataLoader(
            train_dataset,
            batch_size=optimal_config.batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )

        val_dataloader = DataLoader(
            val_dataset,
            batch_size=optimal_config.batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        # 손실 함수 설정
        def criterion_fn(predictions, targets):
            return label_smoothing_cross_entropy(predictions, targets, optimal_config.label_smoothing_factor)
        
        # 옵티마이저 설정
        optimizer = AdamW(
            model.parameters(),
            lr=optimal_config.learning_rate,
            weight_decay=optimal_config.weight_decay
        )

        # 전체 훈련 스텝 계산
        total_steps = len(train_dataloader) * optimal_config.num_epochs // optimal_config.gradient_accumulation_steps

        # 스케줄러 설정 (warmup + linear decay)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=optimal_config.warmup_steps,
            num_training_steps=total_steps
        )

        # 혼합 정밀도 스케일러
        scaler = GradScaler() if torch.cuda.is_available() else None
        
        # 훈련 루프
        best_val_accuracy = 0
        best_epoch = 0
        patience_counter = 0
        
        for epoch in range(optimal_config.num_epochs):
            print(f"\n--- Epoch {epoch + 1}/{optimal_config.num_epochs} ---")
            
            # 훈련
            train_metrics = train_epoch(
                model, train_dataloader, optimizer, scheduler, scaler, 
                fixed_config.device, optimal_config, criterion_fn
            )
            
            # 검증
            val_metrics = validate_epoch(model, val_dataloader, fixed_config.device, criterion_fn)
            
            # WandB 로깅
            if optimal_config.use_wandb:
                wandb.log({
                    "epoch": epoch + 1,
                    "train_loss": train_metrics['loss'],
                    "train_accuracy": train_metrics['accuracy'],
                    "train_f1": train_metrics['f1'],
                    "val_loss": val_metrics['loss'],
                    "val_accuracy": val_metrics['accuracy'],
                    "val_f1": val_metrics['f1'],
                    "learning_rate": scheduler.get_last_lr()[0]
                })
            save_model(model, tokenizer, optimal_config.save_model_path + f"_epoch_{epoch + 1}", epoch + 1, val_metrics['accuracy'])
            # 최고 성능 모델 저장
            if val_metrics['accuracy'] > best_val_accuracy:
                best_val_accuracy = val_metrics['accuracy']
                best_epoch = epoch + 1
                patience_counter = 0
                
                #if optimal_config.save_best_model:
                #   save_model(model, tokenizer, optimal_config.save_model_path, epoch + 1, val_metrics['accuracy'])
            else:
                patience_counter += 1
            
            print(f"Epoch {epoch + 1} 결과:")
            print(f"  Train - Loss: {train_metrics['loss']:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1']:.4f}")
            print(f"  Val   - Loss: {val_metrics['loss']:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
            print(f"  Best Val Accuracy: {best_val_accuracy:.4f} (Epoch {best_epoch})")
            
            # Early stopping
            if patience_counter >= optimal_config.early_stopping_patience:
                print(f"\n⏹️  Early stopping at epoch {epoch + 1} (patience: {optimal_config.early_stopping_patience})")
                break
        
        print(f"\n🎉 훈련 완료!")
        print(f"최고 검증 정확도: {best_val_accuracy:.4f} (Epoch {best_epoch})")
        print(f"모델 저장 위치: {optimal_config.save_model_path}")
        
        # 최종 메트릭 로깅
        if optimal_config.use_wandb:
            wandb.log({
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch,
                "final_train_loss": train_metrics['loss'],
                "final_train_accuracy": train_metrics['accuracy'],
                "final_val_loss": val_metrics['loss'],
                "final_val_accuracy": val_metrics['accuracy']
            })
        
        return best_val_accuracy, best_epoch
        
    except Exception as e:
        print(f"❌ 훈련 중 오류 발생: {str(e)}")
        if optimal_config.use_wandb:
            wandb.log({"error": str(e)})
        raise e
    finally:
        if optimal_config.use_wandb:
            wandb.finish()

print("✅ 최종 훈련 함수 정의 완료")


✅ 최종 훈련 함수 정의 완료


In [15]:
# 최종 모델 훈련 실행
print("🚀 최종 모델 훈련을 시작합니다...")
print(f"프로젝트: {optimal_config.project_name}")
print(f"실행 이름: {RUN_NAME}")
print(f"모델 저장 경로: {optimal_config.save_model_path}")

# 훈련 실행
best_accuracy, best_epoch = train_final_model()

print(f"\n🎉 최종 훈련 완료!")
print(f"최고 검증 정확도: {best_accuracy:.4f}")
print(f"최고 성능 에포크: {best_epoch}")
print(f"모델 저장 위치: {optimal_config.save_model_path}")


🚀 최종 모델 훈련을 시작합니다...
프로젝트: [domain_project]_Final_Training_original
실행 이름: TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
모델 저장 경로: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42


wandb: Currently logged in as: qkdwodus777 (qkdwodus777-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



최종 모델 훈련 시작
learning_rate: 1.00e-05
batch_size: 400
label_smoothing_factor: 0.120
weight_decay: 0.040
num_epochs: 4
augment_ratio: 0.0
모델 저장 경로: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: /data/ephemeral/home/code/Final_Models/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42
로컬 경로에서 모델 로딩 중...
✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!

데이터셋 비율에 따른 샘플링 중...
원본 데이터 분포:
  Original: 136,453개
  Augment: 407,336개
\n샘플링된 데이터 분포:
  Original: 136,453개 (비율: 1.0)
  Augment: 0개 (비율: 0.0)
  총 데이터: 136,453개

--- Epoch 1/4 ---


Training:   0%|          | 0/342 [00:00<?, ?it/s]

Validation:   0%|          | 0/18 [00:00<?, ?it/s]

...

Training:   0%|          | 0/342 [00:00<?, ?it/s]

Validation:   0%|          | 0/18 [00:00<?, ?it/s]

✅ 모델 저장 완료: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42_epoch_4
   Epoch: 4, Val Accuracy: 0.8858
Epoch 4 결과:
  Train - Loss: 0.0951, Accuracy: 0.9466, F1: 0.9462
  Val   - Loss: 0.2188, Accuracy: 0.8858, F1: 0.8847
  Best Val Accuracy: 0.8858 (Epoch 4)

🎉 훈련 완료!
최고 검증 정확도: 0.8858 (Epoch 4)
모델 저장 위치: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42


best_epoch,▁
best_val_accuracy,▁
epoch,▁▃▆█
final_train_accuracy,▁
final_train_loss,▁
final_val_accuracy,▁
final_val_loss,▁
learning_rate,█▆▃▁
train_accuracy,▁▅▇█
train_f1,▁▅▇█
+4,...



🎉 최종 훈련 완료!
최고 검증 정확도: 0.8858
최고 성능 에포크: 4
모델 저장 위치: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42


In [16]:
# 모델 평가 및 테스트 함수
def evaluate_model(model_path, test_data_path=None):
    """저장된 모델을 로드하여 평가하는 함수"""
    print(f"\n📊 모델 평가 시작: {model_path}")
    
    # 모델 로드
    model, tokenizer = model_load_from_local(model_path, fixed_config.model_name)
    if model is None or tokenizer is None:
        print("❌ 모델 로드 실패")
        return None
    
    model = model.to(fixed_config.device)
    model.eval()
    
    # 검증 데이터로 평가
    val_dataset = ReviewDataset(
        val_df["review"],
        val_df["label"],
        tokenizer,
        fixed_config.max_length,
    )
    
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=optimal_config.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # 평가 실행
    val_metrics = validate_epoch(model, val_dataloader, fixed_config.device, 
                                lambda p, t: label_smoothing_cross_entropy(p, t, optimal_config.label_smoothing_factor))
    
    print(f"✅ 모델 평가 완료:")
    print(f"  검증 정확도: {val_metrics['accuracy']:.4f}")
    print(f"  검증 F1 점수: {val_metrics['f1']:.4f}")
    print(f"  검증 손실: {val_metrics['loss']:.4f}")
    
    return val_metrics

# 저장된 모델 평가
if os.path.exists(optimal_config.save_model_path):
    print(f"\n🔍 저장된 모델 평가 중...")
    eval_metrics = evaluate_model(optimal_config.save_model_path)
else:
    print(f"❌ 저장된 모델을 찾을 수 없습니다: {optimal_config.save_model_path}")


❌ 저장된 모델을 찾을 수 없습니다: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42


## 훈련 결과 요약


In [17]:
# 훈련 결과 요약
print("📋 최종 훈련 결과 요약")
print("=" * 50)

print(f"🏆 최고 성능:")
print(f"  검증 정확도: {best_accuracy:.4f}")
print(f"  최고 성능 에포크: {best_epoch}")

print(f"\n⚙️  사용된 하이퍼파라미터:")
print(f"  Learning Rate: {optimal_config.learning_rate:.2e}")
print(f"  Batch Size: {optimal_config.batch_size}")
print(f"  Label Smoothing Factor: {optimal_config.label_smoothing_factor:.3f}")
print(f"  Weight Decay: {optimal_config.weight_decay:.3f}")
print(f"  Epochs: {optimal_config.num_epochs}")

print(f"\n📊 데이터셋 구성:")
print(f"  Augment Ratio: {optimal_config.augment_ratio:.1f}")
print(f"  Original Ratio: {optimal_config.original_ratio:.1f}")

print(f"\n💾 모델 저장 정보:")
print(f"  저장 경로: {optimal_config.save_model_path}")
print(f"  모델 이름: {fixed_config.model_name}")
print(f"  DAPT 사용: {fixed_config.is_DAPT}")

print(f"\n🔗 WandB 정보:")
print(f"  프로젝트: {optimal_config.project_name}")
print(f"  실행 이름: {RUN_NAME}")

print("\n✅ 훈련이 성공적으로 완료되었습니다!")


📋 최종 훈련 결과 요약
🏆 최고 성능:
  검증 정확도: 0.8858
  최고 성능 에포크: 4

⚙️  사용된 하이퍼파라미터:
  Learning Rate: 1.00e-05
  Batch Size: 400
  Label Smoothing Factor: 0.120
  Weight Decay: 0.040
  Epochs: 4

📊 데이터셋 구성:
  Augment Ratio: 0.0
  Original Ratio: 1.0

💾 모델 저장 정보:
  저장 경로: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
  모델 이름: TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42
  DAPT 사용: True

🔗 WandB 정보:
  프로젝트: [domain_project]_Final_Training_original
  실행 이름: TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42

✅ 훈련이 성공적으로 완료되었습니다!


In [18]:
# 모델 사용 예시 함수
def predict_sentiment(text, model_path=None):
    """텍스트의 감정을 예측하는 함수"""
    if model_path is None:
        model_path = optimal_config.save_model_path
    
    # 모델 로드
    model, tokenizer = model_load_from_local(model_path, fixed_config.model_name)
    if model is None or tokenizer is None:
        print("❌ 모델 로드 실패")
        return None
    
    model = model.to(fixed_config.device)
    model.eval()
    
    # 텍스트 토크나이징
    encoding = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=fixed_config.max_length,
        return_tensors="pt"
    )
    
    input_ids = encoding["input_ids"].to(fixed_config.device)
    attention_mask = encoding["attention_mask"].to(fixed_config.device)
    
    # 예측
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(logits, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    # 클래스 라벨 매핑
    class_labels = {0: "매우 부정", 1: "부정", 2: "긍정", 3: "매우 긍정"}
    
    return {
        "text": text,
        "predicted_class": predicted_class,
        "predicted_label": class_labels[predicted_class],
        "confidence": confidence,
        "probabilities": probabilities[0].cpu().numpy()
    }

# 예시 예측
print("\n🔮 모델 예측 예시:")
print("=" * 40)

# 예시 텍스트들
example_texts = [
    "정말 맛있어요! 강력 추천합니다!",
    "별로네요. 기대했던 것보다 아쉬워요.",
    "그냥 그런 것 같아요.",
    "와! 완전 최고예요! 다시 올 거예요!"
]

for text in example_texts:
    result = predict_sentiment(text)
    if result:
        print(f"\n텍스트: {result['text']}")
        print(f"예측: {result['predicted_label']} (신뢰도: {result['confidence']:.3f})")
        print(f"확률 분포: {result['probabilities']}")

print("\n✅ 예측 예시 완료!")



🔮 모델 예측 예시:
로컬 경로에서 토크나이저 로딩 중...
❌ 경로가 존재하지 않습니다: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
❌ 모델 로드 실패
로컬 경로에서 토크나이저 로딩 중...
❌ 경로가 존재하지 않습니다: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
❌ 모델 로드 실패
로컬 경로에서 토크나이저 로딩 중...
❌ 경로가 존재하지 않습니다: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
❌ 모델 로드 실패
로컬 경로에서 토크나이저 로딩 중...
❌ 경로가 존재하지 않습니다: ./Final_Models/original/TAPT_monologg_koelectra-base-v3-discriminator_augX3_best_discriminator_1028_final_training_2025-10-30_05-34-33_RANDOM_42_final_training_2025-10-30_09-07-58_RANDOM_42
❌ 모델 로드 실패

✅ 예측 예시 완료!
